## Exercise - Pricing a Callable Bond

In [1]:
import warnings

import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq

warnings.filterwarnings('ignore')

## 1. Black’s Formula for Bond Options

### 1.1 Price the Vanilla (Non-Callable) Bond

**Theory:**  
The price of a vanilla coupon bond is the present value of all future cash flows, including coupons and the return of principal at maturity.

For a bond with maturity $T=3$ (years) and annual coupons, the cash flows occur at $t=1$, $t=2$, and $t=3$.

The pricing formula is:
$$
P_{vanilla} = \sum_{i=1}^{3} \text{CF}(t_i) \cdot DF(0, t_i)
$$

Where:  
- $\text{CF}(t_i)$ is the cash flow at time $t_i$  
- $DF(0, t_i)$ is the discount factor for time $t_i$ (from the provided data file)

**Cash flows for this bond ($N=100$, $cpn=6\%$):**
- $t=1$: \$6.00
- $t=2$: \$6.00
- $t=3$: \$106.00 (Coupon + Principal)

In [2]:
# Path to file containing the discount curve
data_path = "../data/discount_curve_2025-02-13.xlsx"

# Helper to fetch the discount factor for a specific time to maturity
def get_df(df, t):
    return df.loc[df["ttm"] == t, "discount"].values[0]

def calculate_vanilla_price(path):
    # Read the discount curve from the Excel file into a DataFrame
    df_curve = pd.read_excel(path)

    # Bond details
    face_value = 100
    coupon_rate = 0.06
    coupon = face_value * coupon_rate  # Annual coupon payment

    # Scheduled cash flows and their timings
    payment_times = np.array([1, 2, 3])
    cash_flows = np.array([coupon, coupon, coupon + face_value])

    # Collect discount factors corresponding to the payment times
    discount_factors = np.array([get_df(df_curve, t) for t in payment_times])

    # Bond price is the sum of discounted cash flows
    bond_price = np.dot(cash_flows, discount_factors)
    return bond_price

# Calculate vanilla bond price
vanilla_price = calculate_vanilla_price(data_path)

# Create results DataFrame for display
vanilla_df = pd.DataFrame(
    {"Price": [f"{vanilla_price:.4f}"]},
    index=["Vanilla Bond Price"],
)
vanilla_df_caption = "1.1"
display(vanilla_df.style.set_caption(vanilla_df_caption))

,Price
Vanilla Bond Price,104.9867


### 1.2 Value of the Issuer's Call Option

**Theory:**  
Bond options are typically priced using Black's model (1976), which assumes the forward bond price follows a lognormal distribution. This is preferred over Black-Scholes for fixed income because it naturally incorporates the pull-to-par effect and the term structure of interest rates.

The value of a European call option on a bond is:

$$
C = DF(0, T_{\text{opt}}) \cdot [F \cdot N(d_1) - K \cdot N(d_2)]
$$

Where:

- $F = 103.31$ (Forward Clean Price)  
- $K = 100$ (Clean Strike)  
- $\sigma = 0.0268$ (Volatility)  
- $T_{\text{opt}} = 1.5$ (Option Expiration)  

With

$$
d_1 = \frac{\ln(F/K) + \frac{1}{2} \sigma^2 T_{\text{opt}}}{\sigma \sqrt{T_{\text{opt}}}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T_{\text{opt}}}
$$

In [3]:
# Read discount factor curve from the spreadsheet
df_curve = pd.read_excel(data_path)

def calculate_call_option(F, K, sigma, T_opt, df_topt):
    # Compute d1 and d2 for Black's model, using the given parameters
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T_opt) / (sigma * np.sqrt(T_opt))
    d2 = d1 - sigma * np.sqrt(T_opt)
    # Black's call price
    return df_topt * (F * norm.cdf(d1) - K * norm.cdf(d2))

# Input parameters to option pricing formula
F = 103.31
K = 100
sigma = 0.0268
T_opt = 1.5

# Get discount factor that matches option expiry (1.5 years)
df_1_5 = get_df(df_curve, T_opt)

call_value = calculate_call_option(F, K, sigma, T_opt, df_1_5)

# Create results DataFrame for display
call_df = pd.DataFrame(
    {"Value": [f"{call_value:.4f}"]},
    index=["Issuer's Call Option Value"],
)
call_df_caption = "1.2"
display(call_df.style.set_caption(call_df_caption))

,Value
Issuer's Call Option Value,3.3738


### 1.3 Price of the Callable Bond

**Theory:**  
A callable bond is a package consisting of a long position in a vanilla bond and a short position in a call option (held by the issuer). Therefore, the value of the callable bond is the value of the vanilla bond minus the value of the embedded call option.

$$
\text{Price}_{\text{Callable}} = \text{Price}_{\text{Vanilla}} - \text{Value}_{\text{Call}}
$$

Since the issuer has the right to buy the bond back at par (the strike), the bondholder's potential upside is capped, making the callable bond cheaper than the vanilla bond.

In [4]:
callable_bond_price = vanilla_price - call_value

# Create results DataFrame for display
callable_bond_df = pd.DataFrame(
    {"Value": [f"{callable_bond_price:.4f}"]},
    index=["Callable Bond Price"],
)
callable_bond_df_caption = "1.3"
display(callable_bond_df.style.set_caption(callable_bond_df_caption))

,Value
Callable Bond Price,101.6129


### 1.4 Which assumptions of Black’s formula do we prefer to Black–Scholes for this problem?

**Answer:**
<br>

For pricing bond options, Black's formula is preferred to Black-Scholes due to the following assumptions:

- **Lognormality of the forward price:** Black assumes the *forward* price of the bond is lognormal. Black-Scholes assumes a lognormal *spot* price, which ignores the pull-to-par effect.

- **Correct measure and numeraire:** Black’s framework prices the option under the forward measure associated with the bond maturing at option expiry, which correctly reflects arbitrage-free pricing in fixed income.

These assumptions make Black’s formula more suitable for callable bonds and similar fixed-income options.

### 1.5 Implied Volatility Calculation

**Theory:**  
Implied volatility is the volatility, denoted as $\sigma_{\text{imp}}$, that, when plugged into the Black formula, makes the model price equal to the observed market price ($C_{\text{mkt}} = 3.50$). This value is typically found using numerical root-finding methods.

In [5]:
# Objective function for finding root: returns difference between Black's model price and market price
def black_call_objective(sigma_guess, F, K, T_opt, df_topt, market_price):
    price_guess = calculate_call_option(F, K, sigma_guess, T_opt, df_topt)
    return price_guess - market_price

# Current market price of the call option
market_call_price = 3.50

# Numerically solve for implied volatility.
implied_vol = brentq(
    black_call_objective,
    1e-4,       # Start searching above zero (volatility cannot be negative)
    1e5,        # Arbitrary upper bound
    args=(F, K, T_opt, df_1_5, market_call_price),
)

# Create results DataFrame for display
implied_vol_df = pd.DataFrame(
    {"Value": [f"{implied_vol:.6f} ({implied_vol*100:.4f}%)"]},
    index=["1.5 Implied Volatility"],
)
implied_vol_df_caption = "1.5"
display(implied_vol_df.style.set_caption(implied_vol_df_caption))

,Value
1.5 Implied Volatility,0.030941 (3.0941%)
